# LW3e as a Difference of Fluxes

In [1]:
#For the LaTeX equations (such as eqnarray) in this document to work, include the following in file
#~/.jupyter/_config.yml
#
#parse:
#  myst_enable_extensions:  # default extensions to enable in the myst parser. See https://myst-parser.readthedocs.io/en/latest/using/syntax-optional.html
#     - amsmath
#
#(the default ~/.jupyter/_config.yml will have amsmath commented out)
import numpy as np
from numpy import exp
import sympy as sy
from sympy import latex
from sympy import I   # symbol for sqrt of -1
from sympy.matrices import Matrix, MatrixSymbol
from fractions import Fraction as Fr
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

## Derivation of LW3e
Using spatial gradients from a cubic polynomial

In [2]:
# The sympy symbols
PolyCoeffs = sy.Matrix(sy.symarray("PolyCoeffs", (4,))) # Coefficients a,b,c,d of the polynomial
Psi = sy.Matrix(sy.symarray("Ψ", (4,)), real=True)    # Grid point values at i-2, i-1, i and i+1
polyM = Matrix([[-8,4,-2,1], [-1,1,-1,1], [0,0,0,1], [1,1,1,1]])
PolyCoeffs = polyM.solve(Psi)
Ddx = PolyCoeffs[2]
D2dx2 = 2*PolyCoeffs[1]
D3dx3 = 6*PolyCoeffs[0]
# From the gradients we get the explicit LW3e  increment based on the Courant number
C = sy.symbols("C", real=True)
LW3e = - C*Ddx + C**2/2*D2dx2 - C**3/6*D3dx3
print('LW3e increment is', LW3e)
print('LW3e increment as coefficients of grid points is')
LW3e = sy.collect(sy.expand(LW3e), Psi)
LW3e
#print(latex(LW3e))

LW3e increment is -C**3*(-Ψ_0 + 3*Ψ_1 - 3*Ψ_2 + Ψ_3)/6 + C**2*(Ψ_1 - 2*Ψ_2 + Ψ_3)/2 - C*(Ψ_0/6 - Ψ_1 + Ψ_2/2 + Ψ_3/3)
LW3e increment as coefficients of grid points is


Ψ_0*(C**3/6 - C/6) + Ψ_1*(-C**3/2 + C**2/2 + C) + Ψ_2*(C**3/2 - C**2 - C/2) + Ψ_3*(-C**3/6 + C**2/2 - C/3)

## LW3e as fluxes

In [5]:
# Find Ψ_l and Ψ_r so that LW3e = -Ψ_r + Ψ_l
LW3eR = LW3e # The residual of LW3e after removal of -Ψ_r + Ψ_l
[Psil, Psir] = sy.symbols("Ψ_l, Ψ_r", real=True)
Psil = Psir = sy.S.Zero
for j in range(len(Psi)-1):
    Psil = Psil + Psi[j]*LW3eR.coeff(Psi[j])
    Psir = Psir + Psi[j+1]*LW3eR.coeff(Psi[j])
    LW3eR = sy.collect(sy.expand(LW3e + Psir - Psil), Psi)

print('Residual =', LW3eR, '\nThe right flux is')
Psir

Residual = 0 
The right flux is


Ψ_1*(C**3/6 - C/6) + Ψ_2*(-C**3/3 + C**2/2 + 5*C/6) + Ψ_3*(C**3/6 - C**2/2 + C/3)